In [0]:
# Importando as funções necessárias
from pyspark.sql.functions import max, sum, min, avg, count, countDistinct, col

In [0]:
# Lendo tabelas na camada silver
clientes_df = spark.read.table("ecommerce.silver.clientes")
pedidos_df = spark.read.table("ecommerce.silver.pedidos")
itens_pedidos_df = spark.read.table("ecommerce.silver.itens_pedido")
produtos_df = (
    spark.read.table("ecommerce.silver.produtos")
    .withColumnRenamed("ativo", "produto_ativo")
    .withColumnRenamed("data_criacao", "produto_data_criacao")
)
categorias_df = (
    spark.read.table("ecommerce.silver.categorias")
    .withColumnRenamed("ativo", "categoria_ativo")
    .withColumnRenamed("data_criacao", "categoria_data_criacao")
)
pagamentos_df = spark.read.table("ecommerce.silver.pagamentos")
avaliacoes_df = spark.read.table("ecommerce.silver.avaliacoes")

In [0]:
obt_df = (
    itens_pedidos_df.alias("i")
    .join(pedidos_df.alias("p"), "pedido_id", "left")
    .join(clientes_df.alias("c"), "cliente_id", "left")
    .join(produtos_df.alias("pr"), "produto_id", "left")
    .join(categorias_df.alias("cat"), "categoria_id", "left")
    .join(pagamentos_agg.alias("pg"), "pedido_id", "left")
    .join(avaliacoes_agg.alias("av"), "produto_id", "left")
    .join(metricas_cliente.alias("mc"), "cliente_id", "left")
    .select(
        # Pedido
        col("p.pedido_id"),
        col("p.data_pedido"),
        col("p.status").alias("status_pedido"),
        col("p.valor_total"),

        # Cliente
        col("c.cliente_id"),
        col("c.nome").alias("nome_cliente"),
        col("c.cidade"),
        col("c.estado"),
        col("c.regiao"),

        # Produto
        col("pr.produto_id"),
        col("pr.nome").alias("nome_produto"),
        col("pr.marca"),
        col("pr.preco"),
        col("pr.produto_ativo"),
        col("pr.produto_data_criacao"),

        # Categoria
        col("cat.nome").alias("nome_categoria"),
        col("cat.categoria_ativo"),

        # Item
        col("i.quantidade"),
        col("i.preco_unitario"),
        (col("i.quantidade") * col("i.preco_unitario")).alias("valor_item"),

        # Pagamento
        col("pg.total_pago"),
        col("pg.metodo_pagamento"),
        col("pg.status_pagamento"),

        # Avaliação
        col("av.media_avaliacao"),
        col("av.qtd_avaliacoes"),

        # Métricas Cliente
        col("mc.lifetime_value"),
        col("mc.qtd_pedidos"),
        col("mc.ticket_medio"),
        col("mc.primeiro_pedido"),
        col("mc.ultimo_pedido")
    )
)

In [0]:
# Salvando no Delta
obt_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("ecommerce.silver.obt_ecommerce")